In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




In [2]:
from federatedscope.contrib.data.cikm_cup import call_cikm_cup_data

register_data("cikm_cup", call_cikm_cup_data)

### Register model

In [3]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [4]:
from federatedscope.contrib.trainer.laplacian_trainer_with_domain_with_autoenc_separation import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

### Register metric

### Create the config

#### Set data, model, trainer and metric

In [5]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("federatedscope/gfl/baseline/laplacian_gine_on_cikmcup.yaml")
#cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer_with_domain_separation'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.model.out_channels = 2
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance=1e4
cfg.params.eps=1e-10
cfg.params.p=0.
cfg.params.diff_importance = 0.1
cfg.params.recon_importance = 0.1

#### configure other options

In [6]:
#cfg.use_gpu = True
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'
cfg.federate.method = \
    'Laplacian_domain_sep_gine_class_clients_diff_0_1_recon_0_1_csd_1e4'
#cfg.federate.local_update_steps = 20000000
#cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 200

cfg.federate.client_num = 13
cfg.early_stop.patience = 10000
#cfg.train.optimizer.lr = 0.001
cfg.train.optimizer.weight_decay = 0.0005
cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False
cfg.data.type = "cikm_cup"

In [7]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [8]:
from yacs.config import CfgNode
client_cfg_file = "federatedscope/gfl/baseline/laplacian_gine_on_cikmcup_per_client.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [9]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2022-10-22 09:14:21,958 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [10]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_with_autoenc_client import LaplacianDomainSeparationReconClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2022-10-22 09:14:21,971 (utils:129)INFO: the current machine is at 127.0.1.1
2022-10-22 09:14:21,972 (utils:131)INFO: the current dir is /home/michael/Desktop/backup files/FederatedScope
2022-10-22 09:14:21,973 (utils:132)INFO: the output dir is exp/Laplacian_domain_sep_gine_class_clients_diff_0_1_recon_0_1_csd_1e4_gin_on_cikm_cup_lr0.1_lstep10_/sub_exp_20221022091421
2022-10-22 09:14:21,973 (cikm_cup:58)INFO: Loading CIKMCUP data from /home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competition.
2022-10-22 09:14:21,975 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #1.
2022-10-22 09:14:22,251 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #2.
2022-10-22 09:14:22,269 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #3.
2022-10-22 09:14:22,580 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #4.
2022-10-22 09:14:22,596 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #5.
2022-10-22 09:14:22,615 (cikm_cup:68)INFO: Loading CIKMCUP data for Client #6.
2022-10-22 0

In [ ]:
Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparationReconClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

server params: 
encoder_atom.atom_embedding_list.0.weight


2022-10-22 09:14:25,653 (fed_runner:249)INFO: Server #0 has been set up ... 
2022-10-22 09:14:25,663 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 8
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: 
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transform: []
  type: 

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
local_gnn.convs.0.nn.linears.0.weight
local_gnn.convs.0.nn.linears.0.bias
loca

2022-10-22 09:14:25,835 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 8
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: 
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transform: []
  type: cikm_cup
device: 0
distribute:
  use: False
early_stop:
  delta: 0.0
  improv

Difference train_avg_loss: 125 -> 174
1: 0.539109 -> 0.540
2: 0.614181 -> 0.60414
3: 0.680621 -> 0.675371
4: 0.466614 -> 0.405735
5: 0.64907 -> 0.648837
6: 0.479263 -> 0.470293
7: 0.668663 -> 0.662036
8: 0.438008 -> 0.433998
9: 0.107733 -> 0.118945
10: 0.009074 -> 0.009043
11: 1.128238 -> 1.126667
12: 2.657362 -> 2.661828
13: -> 0.007134


In [ ]:
data

In [ ]:
import os

After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']